In [1]:
import pandas as pd
import numpy as np
import re

## Importing Main Data

Data is imported from e-commerce with unregular product naming. This could be happend due to different administrator, typo, or even banned product(eg. cig product, alcohol, etc). Moreover, we will deal with variaty of SKU (Stock Keeping Unit), and grouping SKU. 

In this situation, the analyst was given the task to report data using standard naming. The strategy of data cleansing will be overcome using pandas and regular expression (Regexp) to mine data, so that the insight will be readyly found.

In [2]:
data_main = pd.read_csv("Vape Sales Data (Jan-Mei 21).csv")
data_main.head()

,Date,Item,Variasi,qty
0,01/01/2021,Paket Stok Bulanan 2,IB,1
1,01/01/2021,Paket Stok Bulanan 2,SF,16
2,01/01/2021,VOKE Cartridge 1pcs,NaN,14
3,01/01/2021,VOKE Cartridge 1pcs,NaN,8
4,01/01/2021,Paket Stok Bulanan 2,Indonesian Kretek,4


## Data Model

Data is modified to combine between varian and main data to observe uniqueness. Furthermore, the all of the item is made in lower case, to ease distinguishing unregular pattern.

In [3]:
data_model = pd.DataFrame()
data_model["item"] = data_main.Item.str.cat(data_main.Variasi,sep="-",na_rep="")
data_model['item'] = [x.lower() for x in data_model['item']]
data_model['qty'] = data_main.qty
data_model.head()

,item,qty
0,paket stok bulanan 2-ib,1
1,paket stok bulanan 2-sf,16
2,voke cartridge 1pcs-,14
3,voke cartridge 1pcs-,8
4,paket stok bulanan 2-indonesian kretek,4


**Unique Product**

Before analyzing full product, firstly unique product will be extracted to an
alyzed how many data that has different pattern

Bundling/Bulanan is Indonesian word to indicate grouping SKU. While every two-character-word is an abbreviation of liquid varian (eg. indonesian kretek -> ik, suffle fruity -> sf, etc).

In [4]:
data_model.item.unique()

array(['paket stok bulanan 2-ib', 'paket stok bulanan 2-sf',
       'voke cartridge 1pcs-', 'paket stok bulanan 2-indonesian kretek',
       'bundling sk 15 ml-ib', 'paket stok bulanan 2-rm',
       'bundling sk 15 ml christmas & nye-sf',
       'bundling sk 15 ml-indonesian kretek', 'voke hygiene mouthpiece-',
       'bundling sk 15 ml-rm', 'bundling sk 15 ml-punch pop',
       'voke berrypop-', 'paket stok bulanan 2-punch pop',
       'bundling sk 15 ml-sf', 'voke liquid c series - ice blast-',
       'voke mango-', 'bundling sk 15 ml - engrave edition-punch pop',
       'voke liquid c series - red mild-', 'red bold 30ml-',
       'bundling sk 15 ml christmas & nye-rm',
       'bundling sk 15 ml christmas & nye-indonesian kretek',
       'bundling sk 15 ml christmas & nye-ib',
       'bundling sk 15 ml - engrave edition-indonesian kretek',
       'bundling sk 15 ml christmas & nye-punch pop',
       'voke liquid c series - java kretek-',
       'bundling sk 15 ml - engrave edition-ib

In [5]:
unique_table = pd.DataFrame(data_model.item.unique())

sku_pattern = r'bundling|bulanan'
unique_table['grouping'] = [bool(re.search(sku_pattern,item)) for item in data_model.item.unique()]
unique_table.head()

,0,grouping
0,paket stok bulanan 2-ib,True
1,paket stok bulanan 2-sf,True
2,voke cartridge 1pcs-,False
3,paket stok bulanan 2-indonesian kretek,True
4,bundling sk 15 ml-ib,True


To ease the classification and identification, better we seperate grouping and single unique SKU

In [6]:
unique_table_group = unique_table[unique_table.grouping==True]
unique_table_single = unique_table[unique_table.grouping==False]

display(unique_table_single.head())
display(unique_table_group.head())

,0,grouping
2,voke cartridge 1pcs-,False
8,voke hygiene mouthpiece-,False
11,voke berrypop-,False
14,voke liquid c series - ice blast-,False
15,voke mango-,False


,0,grouping
0,paket stok bulanan 2-ib,True
1,paket stok bulanan 2-sf,True
3,paket stok bulanan 2-indonesian kretek,True
4,bundling sk 15 ml-ib,True
5,paket stok bulanan 2-rm,True


### Cleansing Grouping Table

The main strategy of cleansing table here, I'm going to use regexp to extract data from each table, to take main item, and varian. The main reason is because there are many unregular words consist in data (eg. red bold -> red boid). Therefore needs more writing library reference/abbreviation to join the table if spreadsheet were use for cleansing. Below how the regexp is useful for cleansing the data:

In [7]:
group_pattern = r'(bundling|bulanan\s?2?)'
liquid_pattern = r'(sf|ib|.ndones.an kretek|punch\s?pop|rm)'
vol_pattern = r'(15\s?ml|30 ?ml)'

#group_code = unique_table_group[0].str.extract(group_pattern)
group_code = pd.DataFrame()
group_code['name'] = unique_table_group[0]
group_code['group_code'] = unique_table_group[0].str.extract(group_pattern)
group_code['liquid_code'] = unique_table_group[0].str.extract(liquid_pattern)
group_code['volume_code'] = unique_table_group[0].str.extract(vol_pattern)

In [9]:
pattern2 = {r"ib|.ce ?b.ast":'ib',
            r'berrypop':'bp',
            r'mango':'mg',
            r'.ndones.an kretek':'ik',
            r'shuffle fruity':'sf',
            r'punch ?pop':'pp',
            r'15\s?ml':'15',
            r'30\s?ml':'',
            np.nan:'',
            r'bundling':'skbd',
            r'bulanan 2':"sb2",
            r'bulanan':'sb'}

group_code.replace(regex=pattern2,inplace=True)
trans = group_code["group_code"].str.cat(group_code['liquid_code'],sep="-")
group_code['std_name'] =trans.str.cat(group_code['volume_code'],sep="")
group_code[['name','std_name']].head()

,name,std_name
0,paket stok sb2-ib,sb2-ib
1,paket stok sb2-sf,sb2-sf
3,paket stok sb2-ik,sb2-ik
4,skbd sk 15-ib,skbd-ib15
5,paket stok sb2-rm,sb2-rm


### Cleansing Single SKU

In [10]:
sku_pattern = r'(sf|ib|.ce ?b.ast|ca.*ge|mo.+ece|ber.+op|lanyard|.ndones.an kretek|punch\s?pop|rm|mango|red ?mi.d|java kretek|red bo.d)'

pattern2 = {r"ib|.ce ?b.ast":'ib',
            r'berrypop':'bp',
            r'red m..d':'rm',
            r'ca.*dge':"ct",
            r'mo.*ece':'mt',
            r'lanyard':'lyd',
            r'jav.*ek':'jk',
            r'mango':'mg',
            r'red ?bo.d':'rb',
            r'.ndones.an kretek':'ik',
            r'shuffle fruity':'sf',
            r'punch ?pop':'pp',
            r'15\s?ml':'15',
            r'30\s?ml':'',
            np.nan:'ib',
            r'bundling':'skbd',
            r'bulanan 2':"sb2",
            r'bulanan':'sb'}

single_code = pd.DataFrame()
single_code['name'] = unique_table_single[0]
single_code['std_name'] = single_code.name.str.extract(sku_pattern)
single_code.std_name.replace(regex=pattern2,inplace=True)
#trans = group_code["group_code"].str.cat(group_code['liquid_code'],sep="-")
#group_code['std_name'] =trans.str.cat(group_code['volume_code'],sep="")
single_code

,name,std_name
2,voke cartridge 1pcs-,ct
8,voke hygiene mouthpiece-,mt
11,voke berrypop-,bp
14,voke liquid c series - ice blast-,ib
15,voke mango-,mg
17,voke liquid c series - red mild-,rm
18,red bold 30ml-,rb
24,voke liquid c series - java kretek-,jk
26,voke llquld red boid-,rb
30,voke catridge 1pcs-,ct


## Stacking Grouping and Single SKU

In [11]:
display(single_code.head())
display(group_code.head())

,name,std_name
2,voke cartridge 1pcs-,ct
8,voke hygiene mouthpiece-,mt
11,voke berrypop-,bp
14,voke liquid c series - ice blast-,ib
15,voke mango-,mg


,name,group_code,liquid_code,volume_code,std_name
0,paket stok sb2-ib,sb2,ib,,sb2-ib
1,paket stok sb2-sf,sb2,sf,,sb2-sf
3,paket stok sb2-ik,sb2,ik,,sb2-ik
4,skbd sk 15-ib,skbd,ib,15,skbd-ib15
5,paket stok sb2-rm,sb2,rm,,sb2-rm


In [12]:
std_code = pd.concat([single_code,group_code[['name','std_name']]])
unique_table.join(std_code.std_name).head()

,0,grouping,std_name
0,paket stok bulanan 2-ib,True,sb2-ib
1,paket stok bulanan 2-sf,True,sb2-sf
2,voke cartridge 1pcs-,False,ct
3,paket stok bulanan 2-indonesian kretek,True,sb2-ik
4,bundling sk 15 ml-ib,True,skbd-ib15


In [18]:
unique_model = unique_table.join(std_code.std_name)[[0,"std_name"]] ##removing grouping table
unique_model

,0,std_name
0,paket stok bulanan 2-ib,sb2-ib
1,paket stok bulanan 2-sf,sb2-sf
2,voke cartridge 1pcs-,ct
3,paket stok bulanan 2-indonesian kretek,sb2-ik
4,bundling sk 15 ml-ib,skbd-ib15
5,paket stok bulanan 2-rm,sb2-rm
6,bundling sk 15 ml christmas & nye-sf,skbd-sf15
7,bundling sk 15 ml-indonesian kretek,skbd-ik15
8,voke hygiene mouthpiece-,mt
9,bundling sk 15 ml-rm,skbd-rm15


This table is supposed to be check first before the join the main data to make sure the data exactly represent of unregular data.

## Finalization

In [17]:
final = data_model.set_index('item').join(unique_model.set_index(0))
final.head(20)

,qty,std_name
bundling sk 15 ml - engrave edition-ib,3,skbd-ib15
bundling sk 15 ml - engrave edition-ib,17,skbd-ib15
bundling sk 15 ml - engrave edition-ib,10,skbd-ib15
bundling sk 15 ml - engrave edition-ib,13,skbd-ib15
bundling sk 15 ml - engrave edition-ib,1,skbd-ib15
bundling sk 15 ml - engrave edition-ib,10,skbd-ib15
bundling sk 15 ml - engrave edition-ib,12,skbd-ib15
bundling sk 15 ml - engrave edition-ib,5,skbd-ib15
bundling sk 15 ml - engrave edition-ib,18,skbd-ib15
bundling sk 15 ml - engrave edition-ib,18,skbd-ib15
